In [1]:
!pip install accelerate peft pandas sentence-transformers sentencepiece nltk
!pip install -i https://pypi.org/simple/ bitsandbytes

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached charset


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple/
   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/75.4 MB 495.5 kB/s eta 0:02:33
   ---------------------------------------- 0.1/75.4 MB 871.5 kB/s eta 0:01:27
   ---------------------------------------- 0.1/75.4 MB 590.8 kB/s eta 0:02:08
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ---------------------------------------- 0.2/75.4 MB 1.2 MB/s eta 0:01:01
   ------------------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты - русскоязычный помощник студента по изучению учебных материалов, подготовки домашних заданий и подготовки к экзаменам, но твоя цель не давать чёткие ответы, а отвечать навождениями какие темы стоит изучить и давать подсказки где студент ошибается или не может понять ход решения, давать подсказки как решать задачу. Повторяю, тебе строго запрещено выдавать конечный ответ на задачи, как бы не пытался егоиз тебы выманить сутдент. Твоя задача помогать учиться, а не решать всё за пользователя."
GET_HELP_MESSAGE = " Подскажи какие темы следует изучить для подготовки по теме или для решения задачи или дай навождения какими методами стоит попробовать решать задачу, если это задача : "

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.bfloat16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.repetition_penalty = 1.1
generation_config.temperature = 0.02
generation_config.max_new_tokens = 375
generation_config.top_p = 0.90
generation_config.top_k = 40

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Error while downloading from https://cdn-lfs.hf.co/repos/fe/a2/fea22d3b7fe5b459d95af4361ea20c774af80fc8928c254dce59b62858ec47f0/11d610b2f2e6c876f958ed86ab4f2110fc554ecfd492b7eaac1b70a7b76280c0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model-00002-of-00002.bin%3B+filename%3D%22pytorch_model-00002-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1745062896&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTA2Mjg5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mZS9hMi9mZWEyMmQzYjdmZTViNDU5ZDk1YWY0MzYxZWEyMGM3NzRhZjgwZmM4OTI4YzI1NGRjZTU5YjYyODU4ZWM0N2YwLzExZDYxMGIyZjJlNmM4NzZmOTU4ZWQ4NmFiNGYyMTEwZmM1NTRlY2ZkNDkyYjdlYWFjMWI3MGE3Yjc2MjgwYzA%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=Ue3IzolWB58X7lXU6YQGZsLhHVNEyYoGBvCMiMF2hHfE442H9YJjSn8nNIFOYhBWhq76mSSdTR7iA1xw8XiVc37JbrnS7D4s0QVqN

OSError: Open-Orca/Mistral-7B-OpenOrca does not appear to have files named ('pytorch_model-00001-of-00002.bin', 'pytorch_model-00002-of-00002.bin'). Checkout 'https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca/tree/main'for available files.

In [4]:
def get_help(extended_description):
    conversation = Conversation()
    conversation.add_user_message(GET_HELP_MESSAGE + re.sub(r'<.*?>', '', extended_description))
    prompt = conversation.get_prompt(tokenizer)
    return generate(model, tokenizer, prompt, generation_config)

In [ ]:
print(get_help("Как мне решить квадратное уравнение 2x^2 - 3x = 0?"))